# LDA - Linear Discriminent Analysis

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset1=pd.read_csv("wine.csv",index_col=None)
df=dataset1
df = pd.get_dummies(df, drop_first=True)
df

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


In [3]:
indep_x=df.iloc[:,0:13].values
dep_y=df.iloc[:,13].values

In [4]:
def split_scaler(indep_x,dep_y):
    x_train,x_test,y_train,y_test = train_test_split(indep_x,dep_y,test_size = 0.2, random_state=0)
    sc=StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    return x_train,x_test,y_train,y_test

def cm_prediction(classifier,x_test,y_test):
    y_pred = classifier.predict(x_test)
    cm = confusion_matrix(y_test,y_pred)
    Accuracy = accuracy_score(y_test,y_pred)
    Report = classification_report(y_test,y_pred)
    return cm,Accuracy,Report

def logistic(x_train,y_train,x_test,y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def svm_linear(x_train,y_train,x_test,y_test):
    classifier = SVC(kernel='linear',random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def svm_nonlinear(x_train,y_train,x_test,y_test):
    classifier = SVC(kernel = 'rbf', random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def knn(x_train,y_train,x_test,y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski')
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def naive(x_train,y_train,x_test,y_test):   
    classifier = GaussianNB()
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def decision(x_train,y_train,x_test,y_test):
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def random(x_train,y_train,x_test,y_test):
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def LDA_results(acclog,accsvml,accsvmnl,accknn,accnav,accdc,accrf):
    LDAdataframe = pd.DataFrame(index = ['LDA'], columns = ['Logistic', 'SVM_l', 'SVM_nl', 'KNN',
                                                                                              'Naive','Decision','Random'])
    for index in LDAdataframe.index:
        LDAdataframe['Logistic'][index] = acclog
        LDAdataframe['SVM_l'][index] = accsvml
        LDAdataframe['SVM_nl'][index] = accsvmnl
        LDAdataframe['KNN'][index] = accknn
        LDAdataframe['Naive'][index] = accnav
        LDAdataframe['Decision'][index] = accdc
        LDAdataframe['Random'][index] = accrf
    return LDAdataframe

In [5]:
acclog =[] 
accsvml = []
accsvmnl = []
accknn = []
accnav = [] 
accdc = []
accrf = []

In [6]:
# Train Test Split

x_train,x_test,y_train,y_test = split_scaler(indep_x,dep_y)

In [7]:
# Applying LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 1)
x_train_lda = lda.fit_transform(x_train,y_train)
x_test_lda = lda.transform(x_test)

In [8]:
x_train.shape[1]

13

In [9]:
cm,Accuracy,Report = logistic(x_train_lda, y_train, x_test_lda, y_test)
acclog.append(Accuracy)

cm,Accuracy,Report = svm_linear(x_train_lda, y_train, x_test_lda, y_test)
accsvml.append(Accuracy)

cm,Accuracy,Report = svm_nonlinear(x_train_lda, y_train, x_test_lda, y_test)
accsvmnl.append(Accuracy)

cm,Accuracy,Report = knn(x_train_lda, y_train, x_test_lda, y_test)
accknn.append(Accuracy)

cm,Accuracy,Report = naive(x_train_lda, y_train, x_test_lda, y_test)
accnav.append(Accuracy)

cm,Accuracy,Report = decision(x_train_lda, y_train, x_test_lda, y_test)
accdc.append(Accuracy)

cm,Accuracy,Report = random(x_train_lda, y_train, x_test_lda, y_test)
accrf.append(Accuracy)

result = LDA_results(acclog[0],accsvml[0],accsvmnl[0],accknn[0],accnav[0],accdc[0],accrf[0])

In [10]:
result

,Logistic,SVM_l,SVM_nl,KNN,Naive,Decision,Random
LDA,0.833333,0.833333,0.833333,0.833333,0.833333,0.888889,0.861111


# As the Accuracy is good in DecisionTree we are going to create a model for DecisionTree